In [3]:
#modules
# import pandas as pd
# import geopandas as gpd
# import contextily as ctx
# from itertools import combinations

import pandas as pd
import geopandas as gpd
from datetime import timedelta
import contextily as ctx
import matplotlib.pyplot as plt

In [5]:
import chardet

with open("dataset_tsmc2014/dataset_TSMC2014_NYC.txt", "rb") as file:
    result = chardet.detect(file.read())
    print(result)

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [6]:
# 1. Chargement des données

# New York data
ny_data = pd.read_csv("dataset_tsmc2014/dataset_TSMC2014_NYC.txt", 
                      sep="\t", header=None, encoding="ISO-8859-1", names=["UserID", "VenueId",  "CategorieId", "CategorieDescription","Latitude",  "Longitude", "Checkins", "Timestamp"] 
                    )

# Tokyo data
tk_data = pd.read_csv("dataset_tsmc2014/dataset_TSMC2014_TKY.txt",
                      sep="\t", header=None, encoding="ISO-8859-1",names=["UserID", "VenueId",  "CategorieId", "CategorieDescription","Latitude",  "Longitude", "Checkins", "Timestamp"]
                    )



In [7]:
# Merge them
data = pd.concat([ny_data, tk_data], ignore_index=True)

In [8]:
# Convert the timestamp column to datetime
data["Timestamp"] = pd.to_datetime(data["Timestamp"], format='%a %b %d %H:%M:%S %z %Y', errors="coerce")


In [11]:
# Data processing

# Sort by place and time
data_sorted = data.sort_values(by=['VenueId', 'Timestamp']).reset_index(drop=True)


In [12]:
# Créer une copie des données pour effectuer une jointure sur les lieux
data_copy = data_sorted.copy()

In [ ]:
# Jointure pour trouver les utilisateurs ayant visité le même lieu
merged = data_sorted.merge(
    data_copy, 
    on="VenueId", 
    suffixes=("_1", "_2")
)

In [61]:
# Identify user interaction
# data = data.head(5)

user_pairs = []

for venue, group in data.groupby("VenueId"):
    group = group.sort_values(by='Timestamp')
    for i, user1 in enumerate(group["UserID"]):
        for j, user2 in enumerate(group["UserID"]):
            if i >= j:
                continue
            time_diff = abs((group.iloc[i]["Timestamp"] - group.iloc[j]["Timestamp"]).total_seconds())
            if(time_diff <= 3600):
                user_pairs.append((user1, user2))

print(user_pairs.__len__())                

KeyboardInterrupt: 

In [35]:
# tk_data.__len__()

# ny_data.__len__()

data
# data.__len__()

,UserID,VenueId,CategorieId,CategorieDescription,Latitude,Longitude,Checkins,Timestamp
25842,746,3fd66200f964a52000e71ee3,4bf58dd8d48988d1e7931735,Music Venue,40.733596,-74.003139,-240,2012-04-20 01:46:59+00:00
49239,421,3fd66200f964a52000e71ee3,4bf58dd8d48988d1e7931735,Music Venue,40.733596,-74.003139,-240,2012-05-04 22:46:10+00:00
49340,417,3fd66200f964a52000e71ee3,4bf58dd8d48988d1e7931735,Music Venue,40.733596,-74.003139,-240,2012-05-04 23:30:08+00:00
62387,665,3fd66200f964a52000e71ee3,4bf58dd8d48988d1e7931735,Music Venue,40.733596,-74.003139,-240,2012-05-12 12:49:15+00:00
99727,908,3fd66200f964a52000e71ee3,4bf58dd8d48988d1e7931735,Music Venue,40.733596,-74.003139,-240,2012-06-07 23:45:52+00:00
...,...,...,...,...,...,...,...,...
799106,2265,511c3737e4b0e21901d26602,4bf58dd8d48988d103941735,Home (private),35.700848,139.563647,540,2013-02-14 01:00:50+00:00
799395,1257,511c5fdfe4b070c9fe9ea395,4eb1daf44b900d56c88a4600,Fair,35.671471,139.770916,540,2013-02-14 03:54:46+00:00
799563,1444,511c753ee4b0ff1c68ecae40,4bf58dd8d48988d12e941735,Government Building,35.531016,139.695786,540,2013-02-14 05:25:29+00:00
799613,1192,511c7b6de4b03c1e65701e3f,4eb1bea83b7b6f98df247e06,Factory,35.582487,139.748526,540,2013-02-14 05:51:52+00:00
